In [33]:
import pandas as pd
import plotly.express as px
from scipy.stats import skew
import pandas as pd
from scipy.stats import pearsonr, chi2_contingency, f_oneway
from sklearn.feature_selection import mutual_info_classif, f_classif
import numpy as np
from scipy.stats import boxcox, skew
from sklearn.model_selection import train_test_split

In [34]:
data = pd.read_csv("D:\\GIT HUB\\GUVI Mini Proj 4\\train_data - train_data.csv")

In [35]:

df_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train.shape
df_test.shape

(26476, 14)

In [ ]:
# Target column is Price

In [36]:
# finding the skewness
cont_col = ['order', 'price']
for i in cont_col:
    skewness_value = skew(df_train[i].values)
    if skewness_value > 1:
        #(Highly Asymmetrical) | Transformation is strongly recommended.
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is < 1 so highly skewed(positive)")
    if skewness_value >= 0.5 and skewness_value <= 1:
        #Transformation is often beneficial, but not always critical
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is < 0.5 so skewed")
    if skewness_value >= -0.5 and skewness_value <= 0.5:
        #n(Approximately Symmetrical) | No transformation typically needed
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is near 0  so not skewed")
    if skewness_value <= -1 :
        #(Highly Asymmetrical) | Transformation is strongly recommended.
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is > -1 so highly skewed(negative)")
    if skewness_value <= -0.5 and skewness_value <= -1:
        #Transformation is often beneficial, but not always critical
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is > -0.5 so skewed")  

The skewness of the order column is: 4.4628 and it is < 1 so highly skewed(positive)
The skewness of the price column is: 0.5254 and it is < 0.5 so skewed


In [37]:
# based on the previous runs (preprocessing for classification)
# Transforming the skewed features using the best transforming methods after analysing the skewness and kurtosis 
df_train['price_transformed'], optimal_lambda = boxcox(df_train['price'])
df_train['order_transformed'] = np.log(df_train['order'])

In [38]:
# Outlier detection

# import pandas as pd
# import numpy as np

# 2. Function to detect outliers using the IQR method (1.5 * IQR Rule)
def detect_iqr_outliers(data, column, factor=1.5):
    """Detects outliers using the IQR method for a given continuous column."""
    
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define the bounds
    lower_bound = Q1 - (factor * IQR)
    upper_bound = Q3 + (factor * IQR)
    
    # Identify the outliers
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    # Report the findings
    print(f"\n--- Outlier Analysis for '{column}' ---")
    print(f"Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"Lower Bound: {lower_bound:.2f}")
    print(f"Upper Bound: {upper_bound:.2f}")
    print(f"Total Outliers Found: {len(outliers)}")
    
    if not outliers.empty:
        # Show the actual outlier values
        print(f"Outlier Values:")
        print(outliers[[column]])
    else:
        print("No outliers found.")
    
    return outliers, lower_bound, upper_bound

# 3. Define the continuous columns
# continuous_features = ['Price', 'Area'] 

# 4. Loop through the continuous columns and apply the detection function
# all_outliers = {}
# for feature in continuous_features:
#     outliers, lower_bound, upper_bound = detect_iqr_outliers(df, feature)
#     all_outliers[feature] = {'outliers': outliers, 'lower': lower_bound, 'upper': upper_bound}

In [39]:
# 4. Loop through the continuous columns and apply the detection function
all_outliers = {} # we did not treat the outlier as the order column failed to pass the hypothesis testing
for feature in cont_col:
    outliers, lower_bound, upper_bound = detect_iqr_outliers(df_train, feature)
    all_outliers[feature] = {'outliers': outliers, 'lower': lower_bound, 'upper': upper_bound}


--- Outlier Analysis for 'order' ---
Q1: 2.00, Q3: 12.00, IQR: 10.00
Lower Bound: -13.00
Upper Bound: 27.00
Total Outliers Found: 7308
Outlier Values:
        order
98961      31
128684     61
71417      35
97096      42
83715      35
...       ...
52256      36
38660      48
106530     72
119346     37
5311       41

[7308 rows x 1 columns]

--- Outlier Analysis for 'price' ---
Q1: 33.00, Q3: 52.00, IQR: 19.00
Lower Bound: 4.50
Upper Bound: 80.50
Total Outliers Found: 1233
Outlier Values:
        price
60138      82
116171     82
56288      82
68443      82
71952      82
...       ...
18070      82
131926     82
52733      82
13545      82
64925      82

[1233 rows x 1 columns]


In [40]:
# 'page2_clothing_model' - is removed from clothing model as it is going to be target encoded
cat_col = ['month', 'day', 'country', 'session_id','page1_main_category', 'colour', 'location', 'model_photography', 'page','price_2']
cont_col = ['order_transformed'] #order
target_col = ['price_transformed'] # price
relevant = []

In [ ]:
#Hypothesis testing

In [41]:
print("1. CONTINUOUS vs. CONTINUOUS") #discarding the 'order' feature as the pearson's r is weak
alpha = 0.05 # Significance Level
# cont_col = ['model_target_encoded']
for i in cont_col:
    correlation, p_value = pearsonr(df_train[i], df_train['price_transformed'])

    print(f"   Pearson's r: {correlation:.3f}")
    print(f"   P-Value: {p_value:.5f}")

    if p_value < alpha:
        print(f"   ✅ **Conclusion: Reject Null Hypothesis.** {i} is significantly linearly related to Target_Cont.")
        relevant.append(i)
    else:
        print(f"   ❌ Conclusion: Fail to Reject Null Hypothesis. {i} is not significantly linearly related to Target_Cont.")
    print("-" * 30)


1. CONTINUOUS vs. CONTINUOUS
   Pearson's r: -0.072
   P-Value: 0.00000
   ✅ **Conclusion: Reject Null Hypothesis.** order_transformed is significantly linearly related to Target_Cont.
------------------------------


In [42]:
## Regression - target - [price]
# ==========================================================
# SCENARIO 2: CONTINUOUS vs. CATEGORICAL
# Test: ANOVA (Analysis of Variance) - General case for F-Test
# Example: Income (Cont. Feature) vs. Target_Cat (Cat. Target)
# Goal: Check if mean Income is different across Default groups (0 vs 1)
# ==========================================================
print("2. CATEGORICAL vs. CONTINUOUS ")
alpha = 0.05 # Significance Level
# Separate the continuous data into groups based on the categorical target
for i in cat_col:
    group_list = []
    for j in data[i].unique():
        group = data[data[i] == j]['price']
        group_list.append(group)


    # The f_oneway test performs ANOVA (for 2+ groups)
    f_statistic, p_value = f_oneway(*group_list)

    print(f"   F-Statistic: {f_statistic:.3f}")
    print(f"   P-Value: {p_value:.5f}")

    if p_value < alpha:
        print(f"   ✅ **Conclusion: Reject Null Hypothesis.** Mean {i} is significantly different between Default groups.")
        relevant.append(i)
    else:
        print(f"   ❌ Conclusion: Fail to Reject Null Hypothesis. Mean {i} is NOT significantly different between Default groups.")
    print("-" * 30)

2. CATEGORICAL vs. CONTINUOUS 
   F-Statistic: 6.221
   P-Value: 0.00005
   ✅ **Conclusion: Reject Null Hypothesis.** Mean month is significantly different between Default groups.
------------------------------
   F-Statistic: 1.255
   P-Value: 0.15867
   ❌ Conclusion: Fail to Reject Null Hypothesis. Mean day is NOT significantly different between Default groups.
------------------------------
   F-Statistic: 14.326
   P-Value: 0.00000
   ✅ **Conclusion: Reject Null Hypothesis.** Mean country is significantly different between Default groups.
------------------------------
   F-Statistic: 1.536
   P-Value: 0.00000
   ✅ **Conclusion: Reject Null Hypothesis.** Mean session_id is significantly different between Default groups.
------------------------------
   F-Statistic: 10981.957
   P-Value: 0.00000
   ✅ **Conclusion: Reject Null Hypothesis.** Mean page1_main_category is significantly different between Default groups.
------------------------------
   F-Statistic: 1805.168
   P-Value: 

In [ ]:
relevant
# ['order_transformed', -- weak - remove
#  'month', - cyclic encode
#  'country', - group + OHE
#  'session_id', -- unique - remove
#  'page1_main_category', - OHE
#  'colour', - group(not req as only one is below thershold and same number of column will be achieved)+ OHE
#  'location', - OHE
#  'model_photography', - OHE
#  'page', - ordinal (leave as it is)
#  'price_2'] - binary
# 'page2_clothing_model' - grouping + target encodeing (hypo test and outlier and skew)+ scaling 

In [ ]:
#Grouping, Target encoding, OHE,Binary,label, cyclic encoding and Scaling

In [43]:
# import pandas as pd
# import numpy as np

# # For demonstration, we'll re-create the DataFrame with high cardinality
# # In your real environment, you would load your existing 'df'.
# np.random.seed(42)
# N = 1000 
# countries_list = ['USA'] * 300 + ['Germany'] * 250 + ['UK'] * 150 + ['France'] * 100 
# num_rare_countries = 43
# rare_country_data = [f'Country_{i}' for i in range(num_rare_countries) for _ in range(5)]
# countries_list += rare_country_data 
# countries_list = countries_list[:N] 
# np.random.shuffle(countries_list)

# models_list = ['Model_A'] * 100 + ['Model_B'] * 80 + ['Model_C'] * 60
# num_rare_models = 213
# rare_model_data = [f'Model_{i}' for i in range(num_rare_models) for _ in range(4)]
# models_list += rare_model_data
# models_list = models_list[:N]
# np.random.shuffle(models_list)

# df = pd.DataFrame({
#     'country': countries_list,
#     'page2_clothing_model': models_list,
#     'target': np.random.randint(0, 2, size=N) 
# })

# --- Main Grouping Function ---
def group_rare_categories(df, column_name, min_percentage=0.01, rare_label='RARE_GROUP'):
    """
    Groups categories in a specified column that fall below a minimum
    percentage threshold into a single 'RARE_GROUP' and returns the updated DataFrame.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - column_name (str): The name of the categorical column to process.
    - min_percentage (float): The threshold (e.g., 0.01 for 1%).
    - rare_label (str): The label to use for the grouped rare categories.
    """
    
    # 1. Calculate the normalized frequency counts
    value_counts = df[column_name].value_counts(normalize=True)
    
    # 2. Identify the categories to be grouped
    rare_categories = value_counts[value_counts < min_percentage].index.tolist()
    
    # Check if grouping is necessary
    if not rare_categories:
        print(f"No categories in '{column_name}' were below the {min_percentage*100:.2f}% threshold. Skipping.")
        return df
    
    # 3. Create the new column name
    new_column_name = f'{column_name}_grouped'
    df[new_column_name] = df[column_name].copy()
    
    # 4. Apply the grouping using .loc for efficient assignment
    df.loc[df[column_name].isin(rare_categories), new_column_name] = rare_label
    
    # 5. Report summary
    original_cardinality = df[column_name].nunique()
    new_cardinality = df[new_column_name].nunique()
    
    print(f"\n--- Grouping Summary for '{column_name}' (Threshold: {min_percentage*100:.2f}%) ---")
    print(f"Original Cardinality: {original_cardinality}")
    print(f"Categories Grouped into '{rare_label}': {len(rare_categories)}")
    print(f"New Cardinality: {new_cardinality}")
    
    return df

In [44]:
# A. Grouping 'country' (Original Card: 46)
# Using a 1% threshold
df = group_rare_categories(df_train, 'country', min_percentage=0.01)

# B. Grouping 'page2_clothing_model' (Original Card: 216)
# Using a slightly lower 0.5% threshold to ensure more rare models are grouped
df = group_rare_categories(df_train, 'page2_clothing_model', min_percentage=0.005)

# 3. Final Verification of the New Columns
print("\n--- Final Grouped Data Head ---")
print(df[['country', 'country_grouped', 'page2_clothing_model', 'page2_clothing_model_grouped']].head())
print("\nFinal Value Counts for 'country_grouped':")
print(df['country_grouped'].value_counts())
print("\nFinal Value Counts for 'page2_clothing_model_grouped':")
print(df['page2_clothing_model_grouped'].value_counts())


--- Grouping Summary for 'country' (Threshold: 1.00%) ---
Original Cardinality: 44
Categories Grouped into 'RARE_GROUP': 40
New Cardinality: 5

--- Grouping Summary for 'page2_clothing_model' (Threshold: 0.50%) ---
Original Cardinality: 216
Categories Grouped into 'RARE_GROUP': 145
New Cardinality: 72

--- Final Grouped Data Head ---
        country country_grouped page2_clothing_model  \
78260        29              29                  C53   
5753         29              29                   B9   
60138         9               9                   A9   
115851       29              29                   A3   
84009        29              29                  P61   

       page2_clothing_model_grouped  
78260                    RARE_GROUP  
5753                             B9  
60138                            A9  
115851                           A3  
84009                    RARE_GROUP  

Final Value Counts for 'country_grouped':
country_grouped
29            85683
9             11615

C:\Users\Hp\AppData\Local\Temp\ipykernel_12128\3883849262.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'RARE_GROUP' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[column_name].isin(rare_categories), new_column_name] = rare_label


In [45]:
df_train = df_train.reset_index(drop=True)

In [46]:
# target encoding and OHE
# import pandas as pd
# import numpy as np
from category_encoders import TargetEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

# Assume 'df' is your DataFrame with 'country_grouped', 'page2_clothing_model_grouped', and 'target'
# (The 'target' is your classification outcome, e.g., 'purchased' or 'converted')

# --- 1. Target Encoding Setup for 'page2_clothing_model_grouped' ---

# We will use StratifiedKFold to ensure the target ratio is preserved in each fold
NFOLDS = 5
kf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)

# Create a placeholder column for the encoded feature
df_train['model_target_encoded'] = np.nan

# Perform Target Encoding via K-Fold Cross-Validation
for train_index, val_index in kf.split(df_train, df_train['price_2']):
    # Initialize TargetEncoder (smoothing is default and recommended)
    encoder = TargetEncoder()
    
    # Fit the encoder ONLY on the K-1 folds (train data)
    encoder.fit(df_train.iloc[train_index]['page2_clothing_model_grouped'], df_train.iloc[train_index]['price_2'])
    
    # Transform the remaining fold (validation data)
    # The transformation uses the means calculated from the train_index
    df_train.loc[val_index, 'model_target_encoded'] = encoder.transform(
        df_train.iloc[val_index]['page2_clothing_model_grouped']
    ).to_numpy().flatten()

# For any new data (when predicting), you must fit the encoder on the ENTIRE training set.
final_model_encoder = TargetEncoder()
final_model_encoder.fit(df_train['page2_clothing_model_grouped'], df_train['price_2'])




,verbose,0
,cols,['page2_clothing_model_grouped']
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'
,min_samples_leaf,20
,smoothing,10
,hierarchy,None


In [47]:
# one hot encoding
ohe=['country_grouped','page1_main_category','colour','location']
for i in ohe:
    # Use pandas get_dummies for simple OHE
    df_ohe = pd.get_dummies(df_train[i], prefix=i, dtype=int)

    # Concatenate the new OHE columns to the main DataFrame
    df_train = pd.concat([df_train.drop(i, axis=1), df_ohe], axis=1)

    # Display the result (replace 'page2_clothing_model_grouped' with the encoded column)
    # print("\n--- Final Encoded Features Head ---")
    # print(data[['country_RARE_GROUP','model_target_encoded']].head())

    
    print(f"One-Hot Encoding ({i}_...) is complete and safe.")

One-Hot Encoding (country_grouped_...) is complete and safe.
One-Hot Encoding (page1_main_category_...) is complete and safe.
One-Hot Encoding (colour_...) is complete and safe.
One-Hot Encoding (location_...) is complete and safe.


In [48]:
#Binary - using label encoding
df_train['model_photography'] = df_train['model_photography'].map({1:0, 2:1})
df_train['model_photography'].value_counts()
df_train['price_2'] = df_train['price_2'].map({1:0, 2:1})
df_train['price_2'].value_counts()

price_2
0    54158
1    51745
Name: count, dtype: int64

In [49]:
# 1. Cyclical Encoding for Month (P=12 for a full year)
P_month = 12
df_train['month_sin'] = np.sin(2 * np.pi * df_train['month'] / P_month)
df_train['month_cos'] = np.cos(2 * np.pi * df_train['month'] / P_month)
# 2. Cyclical Encoding for Day (P=31 for max days in a month)
P_day = 31
df_train['day_sin'] = np.sin(2 * np.pi * df_train['day'] / P_day)
df_train['day_cos'] = np.cos(2 * np.pi * df_train['day'] / P_day)

In [ ]:
#hypothesis testing on the target encoded column

In [25]:
print("1. CONTINUOUS vs. CONTINUOUS") #discarding the 'order' feature as the pearson's r is weak
alpha = 0.05 # Significance Level
cont_col1 = ['model_target_encoded']
for i in cont_col1:
    correlation, p_value = pearsonr(df_train[i], df_train['price_transformed'])

    print(f"   Pearson's r: {correlation:.3f}")
    print(f"   P-Value: {p_value:.5f}")

    if p_value < alpha:
        print(f"   ✅ **Conclusion: Reject Null Hypothesis.** {i} is significantly linearly related to Target_Cont.")
        relevant.append(i)
    else:
        print(f"   ❌ Conclusion: Fail to Reject Null Hypothesis. {i} is not significantly linearly related to Target_Cont.")
    print("-" * 30)


1. CONTINUOUS vs. CONTINUOUS
   Pearson's r: -0.608
   P-Value: 0.00000
   ✅ **Conclusion: Reject Null Hypothesis.** model_target_encoded is significantly linearly related to Target_Cont.
------------------------------


In [26]:
# Finding the skewness in target encoded column
# finding the skewness
# cont_col = ['order', 'price']
for i in cont_col1:
    skewness_value = skew(df_train[i].values)
    if skewness_value > 1:
        #(Highly Asymmetrical) | Transformation is strongly recommended.
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is < 1 so highly skewed(positive)")
    if skewness_value >= 0.5 and skewness_value <= 1:
        #Transformation is often beneficial, but not always critical
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is < 0.5 so skewed")
    if skewness_value >= -0.5 and skewness_value <= 0.5:
        #n(Approximately Symmetrical) | No transformation typically needed
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is near 0  so not skewed")
    if skewness_value <= -1 :
        #(Highly Asymmetrical) | Transformation is strongly recommended.
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is > -1 so highly skewed(negative)")
    if skewness_value <= -0.5 and skewness_value <= -1:
        #Transformation is often beneficial, but not always critical
        print(f"The skewness of the {i} column is: {skewness_value:.4f} and it is > -0.5 so skewed")  

The skewness of the model_target_encoded column is: -0.0278 and it is near 0  so not skewed


In [29]:
# Outlier detection
# 4. Loop through the continuous columns and apply the detection function
all_outliers = {}
for feature in cont_col1:
    outliers, lower_bound, upper_bound = detect_iqr_outliers(df_train, feature)
    all_outliers[feature] = {'outliers': outliers, 'lower': lower_bound, 'upper': upper_bound}


--- Outlier Analysis for 'model_target_encoded' ---
Q1: 1.00, Q3: 2.00, IQR: 1.00
Lower Bound: -0.50
Upper Bound: 3.50
Total Outliers Found: 0
No outliers found.


In [50]:
# scaling
# scaling on the Target encoded column

from sklearn.preprocessing import StandardScaler

# --- 2. Scaling the Target-Encoded Feature ---

# 2a. Instantiate the Scaler
# We use StandardScaler here, which transforms the data to have a mean of 0 and a standard deviation of 1.
scaler = StandardScaler()

# 2b. FIT the Scaler ONLY on the Training Data
# The .fit() method calculates the mean and standard deviation of the 'encoded_feature' 
# in the training set (X_train).
scaler.fit(df_train[['model_target_encoded']])

print("--- Scaler Parameters Learned from df_train ---")
print(f"Mean : {scaler.mean_[0]:.2f}")
print(f"Standard Deviation : {scaler.scale_[0]:.2f}\n")

# 2c. TRANSFORM the Training Data
# Apply the calculated mean and std to transform the training data.
df_train['model_target_encoded_scaled'] = scaler.transform(df_train[['model_target_encoded']])

--- Scaler Parameters Learned from df_train ---
Mean : 1.49
Standard Deviation : 0.40



In [51]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105903 entries, 0 to 105902
Data columns (total 49 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   year                          105903 non-null  int64  
 1   month                         105903 non-null  int64  
 2   day                           105903 non-null  int64  
 3   order                         105903 non-null  int64  
 4   country                       105903 non-null  int64  
 5   session_id                    105903 non-null  int64  
 6   page2_clothing_model          105903 non-null  object 
 7   model_photography             105903 non-null  int64  
 8   price                         105903 non-null  int64  
 9   price_2                       105903 non-null  int64  
 10  page                          105903 non-null  int64  
 11  price_transformed             105903 non-null  float64
 12  order_transformed             105903 non-nul

In [ ]:
# Preparing the test data set

In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26476 entries, 31956 to 34878
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year                  26476 non-null  int64 
 1   month                 26476 non-null  int64 
 2   day                   26476 non-null  int64 
 3   order                 26476 non-null  int64 
 4   country               26476 non-null  int64 
 5   session_id            26476 non-null  int64 
 6   page1_main_category   26476 non-null  int64 
 7   page2_clothing_model  26476 non-null  object
 8   colour                26476 non-null  int64 
 9   location              26476 non-null  int64 
 10  model_photography     26476 non-null  int64 
 11  price                 26476 non-null  int64 
 12  price_2               26476 non-null  int64 
 13  page                  26476 non-null  int64 
dtypes: int64(13), object(1)
memory usage: 3.0+ MB


In [52]:
# target encoding of model in data_test
page2_groups = ['RARE_GROUP', 'B9', 'A9', 'A3', 'P1', 'C56', 'A31', 'C17', 'C49',
       'B31', 'A12', 'C50', 'C12', 'B16', 'A8', 'A5', 'A14', 'C40', 'A17',
       'P16', 'A21', 'A11', 'A7', 'A15', 'P17', 'P2', 'A4', 'B1', 'P33',
       'P15', 'B23', 'C1', 'B12', 'B13', 'B4', 'P48', 'A16', 'B30', 'P4',
       'P12', 'A18', 'P23', 'B10', 'A10', 'A2', 'B17', 'C14', 'B11',
       'A13', 'A1', 'A6', 'C5', 'A33', 'C9', 'B15', 'C7', 'C2', 'B26',
       'B3', 'B32', 'C11', 'P3', 'C8', 'B27', 'P6', 'B2', 'C13', 'B24',
       'B14', 'B19', 'C4', 'B21']
df_test['page2_clothing_model_grouped'] = df_test['page2_clothing_model'].apply(lambda x: 'RARE_GROUP' if x not in page2_groups else x)
# df_test['page2_clothing_model_grouped'].unique()

df_temp = df_train[['page2_clothing_model_grouped','model_target_encoded']]

# df_temp.duplicated().sum()
df_temp.drop_duplicates(inplace= True)
# df_temp.shape
# df_test.shape

# 2. Create the dictionary (key is the merge column, value is the column to transfer)
# Only include the columns you need for the mapping
map_dict = df_temp.set_index('page2_clothing_model_grouped')['model_target_encoded'].to_dict()
# map_dict

# 3. Use the .map() method to apply the new column to df_test
# This is much cleaner and faster than using a lambda/apply for this operation.
df_test['model_target_encoded'] = df_test['page2_clothing_model_grouped'].map(map_dict)

C:\Users\Hp\AppData\Local\Temp\ipykernel_12128\2687283577.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.drop_duplicates(inplace= True)


In [53]:

# 2d. TRANSFORM the Test Data using the SAME Scaler
# This is the critical step to prevent data leakage. We use the mean/std 
# calculated from the training set to scale the test data.
df_test['model_target_encoded_scaled'] = scaler.transform(df_test[['model_target_encoded']])

In [54]:
countries_grouped = [9,24,29,46]
df_test['country_grouped'] = df_test['country'].apply(lambda x: 'RARE_GROUP' if x not in countries_grouped else x)

In [55]:
# one hot encoding
ohe=['country_grouped','page1_main_category','colour','location']
for i in ohe:
    # Use pandas get_dummies for simple OHE
    df_ohe = pd.get_dummies(df_test[i], prefix=i, dtype=int)

    # Concatenate the new OHE columns to the main DataFrame
    df_test = pd.concat([df_test.drop(i, axis=1), df_ohe], axis=1)

    # Display the result (replace 'page2_clothing_model_grouped' with the encoded column)
    # print("\n--- Final Encoded Features Head ---")
    # print(data[['country_RARE_GROUP','model_target_encoded']].head())

    
    print(f"One-Hot Encoding ({i}_...) is complete and safe.")

One-Hot Encoding (country_grouped_...) is complete and safe.
One-Hot Encoding (page1_main_category_...) is complete and safe.
One-Hot Encoding (colour_...) is complete and safe.
One-Hot Encoding (location_...) is complete and safe.


In [56]:
# 1. Cyclical Encoding for Month (P=12 for a full year)
P_month = 12
df_test['month_sin'] = np.sin(2 * np.pi * df_test['month'] / P_month)
df_test['month_cos'] = np.cos(2 * np.pi * df_test['month'] / P_month)
# 2. Cyclical Encoding for Day (P=31 for max days in a month)
P_day = 31
df_test['day_sin'] = np.sin(2 * np.pi * df_test['day'] / P_day)
df_test['day_cos'] = np.cos(2 * np.pi * df_test['day'] / P_day)

In [57]:
#Binary - using label encoding
df_test['model_photography'] = df_test['model_photography'].map({1:0, 2:1})
df_test['model_photography'].value_counts()
df_test['price_2'] = df_test['price_2'].map({1:0, 2:1})
df_test['price_2'].value_counts()

price_2
0    13556
1    12920
Name: count, dtype: int64

In [58]:
# transformation
# Apply the SAME optimal_lambda used in train data to the test data
# optimal_lambda is taken from the transformation performed on the train dataset
df_test['price_transformed'] = boxcox(df_test['price'], lmbda=optimal_lambda) #------------ this is correct

In [ ]:
remove = ['year','month','day','session_id','country','order','price','page2_clothing_model_grouped','page2_clothing_model',
          'model_target_encoded','order_transformed']

df_train.drop(columns=remove, inplace=True)
df_test.drop(columns=remove, inplace=True) # remove the 'order_transformed' from the remove list while running this line

In [80]:
df_test = df_test[df_train.columns]

In [81]:
df_test.columns == df_train.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [82]:
df_test.to_csv('df_test_preprocessed_regression.csv')
df_train.to_csv('df_train_preprocessed_regression.csv')